In [18]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 300)
pd.set_option('max_columns', 200)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

startDate = '20201201'
endDate = '20201202'

readPath = r'\\192.168.10.28\equityTradeLogs'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]

rawOrderLog = []
for thisPath in dataPathLs:
    data = pd.read_csv(thisPath)
    data = data.rename(columns={'mdClockAtArrival': 'caamd'})
    rawOrderLog += [data]
rawOrderLog = pd.concat(rawOrderLog, sort=False)

for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
            'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
    rawOrderLog[col] = rawOrderLog[col].fillna(0).astype('int64')   
rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]

if rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)].shape[0] != 0:
    display('There are accounts with duplicated ticks:')
    display(rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)]\
.groupby(['date', 'colo', 'accCode'])['ars'].size())
    rawOrderLog = rawOrderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep='first')

display('There are ticks with orderDirection 0')
display(rawOrderLog[rawOrderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
            'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])

assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
                                                   .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
try:
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
except:
    print('There are orders with all things same except sdd')
    print(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)])
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId', 'sdd'], keep=False)].shape[0] == 0)
try:
    assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0) 
except:
    print('There are orders with same internalId but different orderId other than accCode 8856 case')
    print(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique()[rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() > 1])

r2 = rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['orderDirection'] != 0)]
r1 = rawOrderLog[(rawOrderLog['accCode'] == 8856) & (rawOrderLog['orderDirection'] != 0)]
r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize']).grouper.group_info[0]
r1 = r1.sort_values(by=['test', 'clockAtArrival'])
r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
r1['vai'] = r1.groupby('test')['vai'].ffill()
r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
r2 = r2.sort_values(by=['test', 'clockAtArrival'])
r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
r2['vai'] = r2.groupby('test')['vai'].ffill()
assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
try:
    assert(sum(r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
except:
    a = r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
    print(pd.merge(r2, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])
orderLog = pd.concat([r1, r2])
del r1
del r2    

orderLog['clock'] = orderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
orderLog['broker'] = orderLog['accCode'] // 100
orderLog["broker"] = np.where(orderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), orderLog['accCode'] // 10000, orderLog["broker"])
orderLog['colo_broker'] = orderLog['colo'].str[:2] + '_' + orderLog['broker'].astype('str')
orderLog['order'] = orderLog.groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'internalId']).grouper.group_info[0]
orderLog['group'] = orderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
orderLog['startClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
orderLog['duration'] = orderLog['clockAtArrival'] - orderLog['startClock']
orderLog['orderPrice'] = orderLog['orderPrice'].apply(lambda x: round(x, 2))
orderLog['tradePrice'] = orderLog['tradePrice'].apply(lambda x: round(x, 2))
orderLog['orderDirection1'] = np.where(orderLog["orderDirection"] == -2, -1, np.where(
    orderLog["orderDirection"] == 2, 1, orderLog["orderDirection"]))
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')


### Assertion 1:  make sure same direction in same date, secid, vai
print('=======================================================================================')
print('1. same date, secid, vai: same direction')
orderLog['directNum'] = orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].transform('nunique')
if len(orderLog[orderLog['directNum'] != 1]) > 0:
    print('opposite direction for same date, same secid, same vai')
    display(orderLog[(orderLog['directNum'] != 1) & (orderLog['updateType'] == 0)].groupby(['accCode'])['orderDirection'].size())
    orderLog = orderLog[orderLog['directNum'] == 1]

assert((orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].nunique() == 1).all() == True)

## Assertion 2:  make sure each account, secid, vai only has one insertion
print('=======================================================================================')
print('2. same date, secid, vai, accCode: one insertion')
a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'colo', 'accCode', 'secid', 'vai'])['clockAtArrival'].count()
if len(a[a > 1]) > 0:
    print('more than one insertion at same time')
    a = a[a>1].reset_index()
    display(a)
    display(a.groupby(['accCode'])['date'].size())
    a = pd.merge(a, orderLog[orderLog['updateType'] == 0][['date', 'colo', 'accCode', 'secid', 'vai', 'order']], on=['date', 'colo', 'accCode', 'secid', 'vai'], how='left')
    orderLog = orderLog[~(orderLog['order'].isin(a['order'].unique()))]

orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                             np.where(orderLog['mse'] == 100, 1, 0), np.nan)
orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()

placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
print('%.2f%% SZE orders triggered by msg data'%(placeSZE[placeSZE['isMsg'] == 1].shape[0]/placeSZE.shape[0]*100))


### Assertion 3:  check IPO stocks selling status
print('=======================================================================================')
print('3. IPO stocks selling (ars = 301, 302)')
if orderLog[orderLog['ars'].isin([301, 302])].shape[0] != 0:
    kk = orderLog[orderLog['ars'].isin([301, 302])]
    print(kk)
    try:
        assert(kk[kk['orderDirection1'] == 1].shape[0] == 0)
        print('we only sell, never buy')
    except:
        print('There are IPO buy side orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk[kk['orderDirection1'] == 1])
    kk1 = kk[kk['updateType'] == 0]
    kk1 = kk1.sort_values(by=['accCode', 'secid','clockAtArrival'])
    kk1['diff'] = kk1.groupby(['accCode', 'secid'])['clockAtArrival'].apply(lambda x: x-x.shift(1))
    kk1['diff'] = kk1['diff'].fillna(0)
    try:
        assert(kk1[kk1['diff'] < 10e6].shape[0] == 0)
        print('for each stock in the same account, there is no insertion within 10 seconds of the previous insertion')
    except:
        print('There are insertion within 10 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk1[kk1['diff'] < 10e6])
    kk2 = kk[(kk['updateType'] == 1)]
    try:
        assert(kk2[kk2['duration'] < 3e6].shape[0] == 0)
        print('for each stock in the same account, the cancellation of an order happens more than 3 seconds after the insertion')
    except:
        print('There are cancellation within 3 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk2[kk2['duration'] < 3e6])


### Assertion 4: check updateType == 7 orders, make sure updateType == 7 orders < 20 per account, < 100 in total
print('=======================================================================================')
print('4. updateType 7 orders')
if orderLog[orderLog['updateType'] == 7].shape[0] != 0:
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().max() < 20)
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().sum() < 100)

### Assertion 5: check updateType == 6 orders, make sure updateType == 6 orders < 5% per account
print('=======================================================================================')
print('5. updateType 6 orders')
k1 = orderLog[orderLog['updateType'] == 6].groupby(['date', 'accCode'])['order'].nunique().reset_index()
k2 = orderLog.groupby(['date', 'accCode'])['order'].nunique().reset_index()
k = pd.merge(k1, k2, on=['date', 'accCode'], how='left')
k['prob'] = k['order_x']/k['order_y']
try:
    assert(sum(k['prob'] >= 0.05) == 0)
except:
    print('There are accounts with more than 5% updateType 6 orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(k[k['prob'] >= 0.05])

### Assertion 6: check CYB orders, make sure all CYB stocks have absOrderSize < 30w
print('=======================================================================================')
print('6. CYB stocks order size < 30w')
try:
    cyb = orderLog[(orderLog['secid'] >= 2300000) & (orderLog['updateType'] == 0)].groupby('date')['absOrderSize'].max().reset_index()
    assert(cyb[cyb['absOrderSize'] > 300000].shape[0] == 0)
except:
    print('CYB stocks total absOrderSize >= 30w!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    
     
### Assertion 7:  make sure there is no unexpected updateType 
print('=======================================================================================')
print('7. unexpected updateType')
def getTuple(x):
    return tuple(i for i in x)

checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 1, 4), (0, 4, 1, 4), (0, 2, 2, 4), (0, 4, 2, 4), (0, 2, 2, 1, 4), (0, 2, 2, 4, 1, 4)]),0,
                     np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 1, 4, 3),
                                                               (0, 2, 2, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3),
                                                               (0, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 1)]), 1,
                     np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3), (0, 2, 3), (0, 3), (0, 1, 3), (0, ), (0, 2), (0, 2, 1), (0, 2, 2)]), 2, 3)))

orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

### Assertion 8:  make sure status==0 got all traded
print('=======================================================================================')
print('8. status == 0: all traded')
a = orderLog[orderLog['status'] == 0]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in total trade, any fill != total cases')
display(a[a['filled'] != a['total']])
if a[a['filled'] != a['total']].shape[0] > 0:
    removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 9:  make sure status==1 got partial traded
print('=======================================================================================')
print('9. status == 1: partial traded')
a = orderLog[orderLog['status'] == 1]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in partial trade, any fill >= total or fill is 0 cases for updateType 4')
display(a[(a['filled'] >= a['total']) | (a['filled'] == 0)])
if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
    removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 10: make sure no cancellation within 1 sec
print('=======================================================================================')
print('10. no cancellation within 1 sec')
a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
print('any cancellation within 1 sec')
display(a)
if a.shape[0] > 0:
    removeOrderLs = a['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


### Assertion 11: make sure no order has shares > 80w or notional > 800w
print('=======================================================================================')
print('11. Orders with size > 80w or notional > 800w')
orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']
if orderLog[orderLog['absOrderSize'] > 800000].shape[0] > 0:
    print('some order quantity are > 80w')
    print(orderLog[orderLog['absOrderSize'] > 800000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['absOrderSize'] > 800000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                         'orderNtl', 'orderDirection', 'clock', 'order']])
            
if orderLog[orderLog['orderNtl'] > 8000000].shape[0] > 0:
    print('some order ntl are > 800w')
    print(orderLog[orderLog['orderNtl'] > 8000000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['orderNtl'] > 8000000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                      'orderNtl', 'orderDirection', 'clock', 'order', "updateType", 
                                                      "tradePrice", "absOrderSizeCumFilled", "absFilledThisUpdate"]])

removeOrderLs = list(set(orderLog[orderLog['absOrderSize'] > 800000]['order'].unique()) | set(orderLog[orderLog['orderNtl'] > 8000000]['order'].unique()))
orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

orderLog["mrsb90"] = orderLog.groupby(['order'])['mrsb90'].transform('first')
orderLog["mrss90"] = orderLog.groupby(['order'])['mrss90'].transform('first')
orderLog["aaa"] = orderLog.groupby(['order'])['aaa'].transform('first')

orderLog['m1'] = orderLog['mrstaat'].apply(lambda x: x - (x // 10000) * 10000)
orderLog['m2'] = orderLog['mrstauc'].apply(lambda x: x - (x // 10000) * 10000)
if orderLog[orderLog['mrsb90'] == '-'].shape[0] != 0:
    display(orderLog[orderLog['mrsb90'] == '-'])
orderLog = orderLog[orderLog['mrsb90'] != '-']
orderLog['mrsb90'] = orderLog['mrsb90'].astype(float)
if orderLog[orderLog['aaa'] == '-'].shape[0] != 0:
    display(orderLog[orderLog['aaa'] == '-'])
orderLog = orderLog[orderLog['aaa'] != '-']
orderLog['aaa'] = orderLog['aaa'].astype(float)
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm1']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm1']    

orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
orderLog["mrstaat"] = orderLog.groupby(['order'])['mrstaat'].transform('first')
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
orderLog["mrstauc"] = orderLog.groupby(['order'])['mrstauc'].transform('first')
orderLog = orderLog[~orderLog['ars'].isnull()]
orderLog['sta'] = np.where(orderLog['mrstaat'] == 1000, '1. staone', np.where(
orderLog['mrstaat'] == 3000, '2. statwo', np.where(
orderLog['mrstaat'].isin([11000, 13000]), '3. sta300', 'else')))
display(orderLog[orderLog['sta'] == 'else'][['date', 'secid', 'colo', 'accCode', 'updateType', 'vai', 'absOrderSize', 'orderDirection', 'mrstaat', 'mrstauc', 'sdd', 'ApplSeqNum', 'sta']])
orderLog = orderLog[orderLog['mrstaat'].isin([11000, 13000, 1000, 3000])]
display(orderLog[orderLog['updateType'] == 0].groupby(['mrstaat', 'mrstauc'])['date'].size())

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (54,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


'There are ticks with orderDirection 0'

,date,colo,accCode,secid,vai,updateType,sdd,orderDirection,absOrderSize,internalId,orderId
356179,20201201,zs_52_06,5269,2002054,-1,1,53519.0,0,0,-1.0,1.565274e+06
364036,20201201,zs_54_01,5474,2002090,-1,7,39529.0,0,0,-1.0,8.689054e+08
383430,20201201,zs_52_09,5291,2002191,-1,1,47233.0,0,0,-1.0,1.105797e+06
472188,20201201,zs_54_01,5474,2002729,-1,1,39751.0,0,0,-1.0,8.689061e+08
602842,20201201,zs_52_06,5269,2300402,-1,1,35269.0,0,0,-1.0,2.996450e+05
667032,20201201,zs_54_01,5474,2300635,-1,1,47774.0,0,0,-1.0,8.689105e+08
690343,20201201,zs_54_01,5474,2300708,-1,1,52912.0,0,0,-1.0,8.689160e+08
993891,20201202,zs_96_02,9655,2000100,-1,7,39512.0,0,0,-1.0,-1.000000e+00
1008845,20201202,zs_64_01,6480,2000554,-1,1,53781.0,0,0,-1.0,1.373330e+05
1039368,20201202,zs_52_06,5269,2000893,-1,1,38864.0,0,0,-1.0,7.671610e+05


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1. same date, secid, vai: same direction
opposite direction for same date, same secid, same vai


...

10. no cancellation within 1 sec
any cancellation within 1 sec


,ApplSeqNum,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,ars,caamd,cancellationPending,cfe,clock,clockAtArrival,cumSharesBought,cumSharesBuyInserted,cumSharesSellInserted,cumSharesSold,date,finalState,gfe,hee,insertedShortOrder,insertionPending,internalId,inv_L,inv_L0,inv_S,inv_S0,l4tr,locateShares,locateSharesTotal,mfe,mra100,mrb100,mrm,mrm25,mrmum,mrrlma,mrsb300,mrsb90,mrss300,mrss90,mrstaat,mrstauc,mrstaum,mrv,ms,mse,mt,mta,mv,orderDirection,orderId,orderOutstanding,orderPrice,orderSysId,sdd,secid,sequenceNo,session,threadId,totalActions,totalCanceled,tradeId,tradePrice,underlyingIndex,updateType,vai,zipFile,colo,test,broker,colo_broker,order,group,startClock,duration,orderDirection1,directNum,isMsg,status


11. Orders with size > 80w or notional > 800w


,date,secid,colo,accCode,updateType,vai,absOrderSize,orderDirection,mrstaat,mrstauc,sdd,ApplSeqNum,sta
177632,20201201,1603045,zt_52_10,529001,0,0.0,500,-1,NaN,NaN,145636000.0,NaN,else
258124,20201201,1603811,zs_96_08,974101,0,0.0,200,-1,NaN,NaN,145637000.0,NaN,else
344337,20201201,2000961,zs_88_04,8967,0,34243047.0,6200,1,NaN,NaN,145255020.0,NaN,else
625404,20201201,2300487,zs_54_01,5456,0,1787257.0,500,-1,NaN,NaN,145348400.0,NaN,else
982958,20201202,1688557,zs_96_08,9685,0,302.0,1021,-1,0.0,0.0,134154000.0,0.0,else
982959,20201202,1688557,zt_96_09,966501,0,302.0,1021,-1,0.0,0.0,134154000.0,0.0,else
982960,20201202,1688557,zt_96_09,966501,2,302.0,1021,-1,0.0,0.0,-1.0,0.0,else
982961,20201202,1688557,zt_96_09,966501,4,302.0,1021,-1,0.0,0.0,-1.0,0.0,else
982962,20201202,1688557,zt_96_09,966501,4,302.0,1021,-1,0.0,0.0,-1.0,0.0,else
982963,20201202,1688578,zt_52_10,522401,0,302.0,4729,-1,0.0,0.0,134257000.0,0.0,else


mrstaat  mrstauc
1000.0   0.0         52744
3000.0   1000.0     229501
         2000.0      16566
         3000.0       6771
11000.0  10000.0      1477
         20000.0       124
         30000.0        96
13000.0  11000.0     28686
         12000.0      1106
         13000.0       347
         21000.0       903
         22000.0       953
         23000.0       162
         31000.0       401
         32000.0       337
         33000.0       426
Name: date, dtype: int64

In [2]:
IF_group = pd.read_table(r'E:\stockGroup_IF.txt', header=None)
IC_group = pd.read_table(r'E:\stockGroup_IC.txt', header=None)
CSI1000_group = pd.read_table(r'E:\stockGroup_CSI1000.txt', header=None)
IF_group1 = IF_group.iloc[0, :].values[0][1:-2].split(', ')
IF_group2 = IF_group.iloc[1, :].values[0][1:-2].split(', ')
IC_group1 = IC_group.iloc[0, :].values[0][1:-2].split(', ')
IC_group2 = IC_group.iloc[1, :].values[0][1:-2].split(', ')
CSI1000_group1 = CSI1000_group.iloc[0, :].values[0][1:-2].split(', ')
CSI1000_group2 = CSI1000_group.iloc[1, :].values[0][1:-2].split(', ')
assert(len(IF_group1) == len(IF_group2))
assert(len(IC_group1) == len(IC_group2))
assert(len(CSI1000_group1) == len(CSI1000_group2))

test1 = orderLog[orderLog['secid'].isin(IF_group1 + IF_group2)]
test1['group_tag'] = np.where(test1['secid'].isin(IF_group1), 1, 2)
display(test1[(test1['updateType'] == 0)].groupby(['colo', 'accCode', 'group_tag'])['ars'].unique())
display(test1[(test1['updateType'] == 0) & (test1['accCode'] == 9461)].groupby(['colo', 'accCode', 'group_tag'])['secid'].nunique())
test2 = orderLog[orderLog['secid'].isin(IC_group1 + IC_group2)]
test2['group_tag'] = np.where(test2['secid'].isin(IC_group1), 1, 2)
display(test2[(test2['updateType'] == 0)].groupby(['colo', 'accCode', 'group_tag'])['ars'].unique())
display(test2[(test2['updateType'] == 0) & (test2['accCode'] == 9441)].groupby(['colo', 'accCode', 'group_tag'])['secid'].nunique())
test3 = orderLog[orderLog['secid'].isin(CSI1000_group1 + CSI1000_group2)]
test3['group_tag'] = np.where(test3['secid'].isin(CSI1000_group1), 1, 2)
display(test3[(test3['updateType'] == 0)].groupby(['colo', 'accCode', 'group_tag'])['ars'].unique())
display(test3[(test3['updateType'] == 0) & (test3['accCode'].isin([5456, 9758]))].groupby(['colo', 'accCode', 'group_tag'])['secid'].nunique())

test = test3[test3['accCode'] == 5456]

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


colo      accCode  group_tag
zs_52_06  5384     1            [1.0]
                   2            [1.0]
zs_52_08  5281     1            [1.0]
                   2            [1.0]
zs_52_09  5264     1            [1.0]
                   2            [1.0]
zs_88_04  8967     1            [1.0]
                   2            [1.0]
          892402   1            [1.0]
                   2            [1.0]
zs_94_03  9461     1            [2.0]
                   2            [1.0]
zs_94_05  9435     1            [1.0]
                   2            [1.0]
zs_96_02  9655     1            [1.0]
                   2            [1.0]
zs_96_06  9765     1            [1.0]
                   2            [1.0]
zt_88_06  8970     1            [1.0]
                   2            [1.0]
          8971     1            [1.0]
                   2            [1.0]
Name: ars, dtype: object

colo      accCode  group_tag
zs_94_03  9461     1            29
                   2            32
Name: secid, dtype: int64

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


colo      accCode  group_tag
zs_52_06  5332     1                 [1.0]
                   2                 [1.0]
zs_52_08  5229     1                 [1.0]
                   2                 [1.0]
          5276     1                 [1.0]
                   2                 [1.0]
          5292     1                 [1.0]
                   2                 [1.0]
          5377     1                 [1.0]
                   2                 [1.0]
          5386     1                 [1.0]
                   2                 [1.0]
zs_52_09  5289     1                 [1.0]
                   2                 [1.0]
          5291     1                 [1.0]
                   2                 [1.0]
zs_88_04  8856     1                 [1.0]
                   2                 [1.0]
          8924     1                 [1.0]
                   2                 [1.0]
          896702   1                 [1.0]
                   2                 [1.0]
zs_94_03  9448     1     

colo      accCode  group_tag
zt_94_06  9441     1            37
                   2            29
Name: secid, dtype: int64

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


colo      accCode  group_tag
zs_52_06  5232     1                      [1.0]
                   2                      [1.0]
          5275     1                      [1.0]
                   2                      [1.0]
          5287     1                      [1.0]
                   2                      [1.0]
zs_54_01  5456     1                 [3.0, 2.0]
                   2                      [1.0]
zs_66_01  6623     1                      [1.0]
                   2                      [1.0]
          6627     1                      [1.0]
                   2                      [1.0]
          6683     1                      [1.0]
                   2                      [1.0]
zs_88_04  8865     1                      [1.0]
                   2                      [1.0]
zs_92_02  9243     1                      [1.0]
                   2                      [1.0]
zs_94_05  9471     1                      [1.0]
                   2                      [1.0]
zs_96_06  9

colo      accCode  group_tag
zs_54_01  5456     1            87
                   2            79
zs_96_06  9758     1            84
                   2            83
Name: secid, dtype: int64

In [17]:
orderLog[(orderLog['ars'] == 3) & (orderLog['accCode'] == 966301) & (orderLog['updateType'] == 0)][['colo', 'accCode', 'vai', 'ars', 'orderDirection', 'absOrderSize', 'orderPrice', 'absFilledThisUpdate', 'mrstaat', 'mrstauc', 'sdd', 'mra100', 'mrb100']]

,colo,accCode,vai,ars,orderDirection,absOrderSize,orderPrice,absFilledThisUpdate,mrstaat,mrstauc,sdd,mra100,mrb100
50101,zt_96_01,966301,17204900.0,3.0,-1,100,10.38,0,1000.0,0.0,131542000.0,1038.0,1037.0
52185,zt_96_01,966301,58347512.0,3.0,-1,7100,8.66,0,1000.0,0.0,145534000.0,866.0,865.0
66258,zt_96_01,966301,58064149.0,3.0,-1,200,4.70,0,1000.0,0.0,130133000.0,470.0,469.0
87425,zt_96_01,966301,24637769.0,3.0,-1,2400,6.75,0,1000.0,0.0,130133000.0,675.0,674.0
87428,zt_96_01,966301,24697369.0,3.0,-1,1000,6.75,0,1000.0,0.0,130142000.0,675.0,674.0
88288,zt_96_01,966301,6063831.0,3.0,1,100,8.27,0,1000.0,0.0,141546000.0,828.0,827.0
88336,zt_96_01,966301,8564231.0,3.0,1,1000,8.16,0,1000.0,0.0,144530000.0,817.0,816.0
89333,zt_96_01,966301,4728268.0,3.0,1,100,4.37,0,1000.0,0.0,141558000.0,438.0,437.0
101512,zt_96_01,966301,2818031.0,3.0,1,100,5.23,0,1000.0,0.0,130143000.0,524.0,523.0
113009,zt_96_01,966301,3074931.0,3.0,1,100,4.65,0,1000.0,0.0,141613000.0,466.0,465.0


In [5]:
orderLog[orderLog['ars'] == 3]['accCode'].unique()

array([  8854, 966301,   5456,   9441], dtype=int64)

In [90]:
test['index_name'] = 'CSI1000'
test['order_type'] = np.where(test['ars'] == 1, 'take', np.where(
test['ars'] == 2, 'improved-NBBO make', 'on-NBBO make'))
test['Side'] = np.where(test['orderDirection'] == 1, 'Buy', 'Sell')
test['duration'] = np.where(test['date'] == 20201201, '20201201-20201201', '20201202-20201202')
f1 = test[(test['updateType'] == 0)].groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['orderNtl'].sum().reset_index()
f2 = test[(test['updateType'] == 4)].groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['tradeNtl'].sum().reset_index()
f = pd.merge(f1, f2, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
f['fill_rate'] = f['tradeNtl'] / f['orderNtl']
f1 = test[(test['updateType'] == 0)].groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['order'].nunique().reset_index()
f2 = test[(test['updateType'] == 3)].groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['order'].nunique().reset_index()
ff = pd.merge(f1, f2, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
ff['cancel_rate'] = ff['order_y'] / ff['order_x']
f = pd.merge(f, ff[['duration', 'Side', 'order_type', 'cancel_rate', 'accCode', 'index_name']], on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
test['cancel_duration_med'] = test['clockAtArrival'] - test['startClock']
test['cancel_duration_90p'] = test['clockAtArrival'] - test['startClock']
ff = test[test['updateType'] == 3].groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['cancel_duration_med'].median().reset_index()
f = pd.merge(f, ff, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
ff = test[test['updateType'] == 3].groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['cancel_duration_90p'].quantile(0.9).reset_index()
f = pd.merge(f, ff, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
test1 = test[test['updateType'] == 4]
test1['mid'] = (test1['mra100'] + test1['mrb100'])/200
test1['tp'] = (test1['mrm'] + 1) * test1['mid']
test1['mrm_return'] = np.where(test1['Side'] == 'Sell', test1['tradePrice']/test1['tp'] - 1, test1['tp']/test1['tradePrice'] - 1)
test1['sumNtl'] = test1.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['tradeNtl'].transform(sum)
test1['sum_return'] = test1['mrm_return'] * test1['tradeNtl']
test1['sum_return'] = test1.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['sum_return'].transform(sum)
test1['mrm_return'] = test1['sum_return'] / test1['sumNtl']
ff = test1.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['mrm_return'].first().reset_index()
f = pd.merge(f, ff, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
from IPython.display import display, HTML
HTML(f.groupby(['duration', 'Side',  'index_name', 'accCode', 'order_type']).first().to_html())

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [105]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 100)
pd.set_option('max_columns', 100)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

import pymongo
import pandas as pd
import numpy as np
import pickle
import datetime
import time
import gzip
import lzma
import pytz

def DB(host, db_name, user, passwd):
    auth_db = db_name if user not in ('admin', 'root') else 'admin'
    url = 'mongodb://%s:%s@%s/?authSource=%s' % (user, passwd, host, auth_db)
    client = pymongo.MongoClient(url, maxPoolSize=None)
    db = client[db_name]
    return db

def read_memb_daily(db, name, start_date=None, end_date=None, skey=None, index_id=None, interval=None, col=None, return_sdi=True):
    collection = db[name]
    # Build projection
    prj = {'_id': 0}
    if col is not None:
        if return_sdi:
            col = ['skey', 'date', 'interval'] + col
        for col_name in col:
            prj[col_name] = 1

    # Build query
    query = {}
    if skey is not None:
        query['skey'] = {'$in': skey}
    if index_id is not None:
        query['index_id'] = {'$in': index_id}
    if interval is not None:
        query['interval'] = {'$in': interval}
    if start_date is not None:
        if end_date is not None:
            query['date'] = {'$gte': start_date, '$lte': end_date}
        else:
            query['date'] = {'$gte': start_date}
    elif end_date is not None:
        query['date'] = {'$lte': end_date}

    # Load data
    cur = collection.find(query, prj)
    df = pd.DataFrame.from_records(cur)
    if df.empty:
        df = pd.DataFrame()
    else:
        df = df.sort_values(by=['date', 'index_id', 'skey'])
    return df    

def build_query(start_date=None, end_date=None, index_id=None):
    query = {}
    def parse_date(x):
        if type(x) == int:
            return x
        elif type(x) == str:
            if len(x) != 8:
                raise Exception("date must be YYYYMMDD format")
            return int(x)
        elif type(x) == datetime.datetime or type(x) == datetime.date:
            return x.strftime("%Y%m%d").astype(int)
        else:
            raise Exception("invalid date type: " + str(type(x)))
    if start_date is not None or end_date is not None:
        query['date'] = {}
        if start_date is not None:
            query['date']['$gte'] = parse_date(start_date)
        if end_date is not None:
            query['date']['$lte'] = parse_date(end_date)
    def parse_symbol(x):
        if type(x) == int:
            return x
        else:
            return int(x)
    if index_id:
        if type(index_id) == list or type(index_id) == tuple:
            query['index_id'] = {'$in': [parse_symbol(x) for x in index_id]}
        else:
            query['index_id'] = parse_symbol(index_id)
    return query

def write_memb_data(db, name, df):
    collection = db[name]
    df1 = []
    for symbol in df['index_id'].unique():
        if symbol in collection.distinct('index_id'):
            symbol = int(symbol)
            m_ax = pd.DataFrame.from_records(collection.find({'index_id':{'$in':[symbol]}}).sort([('date',-1)]).skip(0).limit(1))['date'].values[0]
            df2 = df[(df['index_id'] == symbol) & (df['date'] > m_ax)]
            print(df2)
            df1 += [df2]
        else:
            print(symbol)
            df2 = df[(df['index_id'] == symbol)]
            print(df2)
            df1 += [df2]
    df1 = pd.concat(df1).reset_index(drop=True)
    df1 = df1.to_dict('records')
    collection.insert_many(df1) 

def delete_memb_data(db, name, start_date=None, end_date=None, index_id=None):
    collection = db[name]
    query = build_query(start_date, end_date, index_id)
    if not query:
        print('cannot delete the whole table')
        return None
    collection.delete_many(query)  


database_name = 'com_md_eq_cn'
user = "zhenyuy"
password = "bnONBrzSMGoE"

pd.set_option('max_columns', 200)
db1 = DB("192.168.10.178", database_name, user, password)

import os
import glob
import datetime
import numpy as np
import pandas as pd
import pickle
from IPython.display import display, HTML
# 上次遗留问题：调整图间间距
# 调整标签位置，不要和图中重合
startDate = '20201201'
endDate = '20201202'

readPath = "L:\\orderLog\\ret\\***"
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
df = []
add = read_memb_daily(db1, 'index_memb', int(startDate), int(endDate), index_id=[1000300, 1000905, 1000852, 1000985])
add = add.rename(columns={'skey':'secid'})

dateLs = np.sort(dateLs)
dataPathLs = np.sort(dataPathLs)
for thisDate, thisPath in zip(dateLs, dataPathLs):
    data = pd.read_pickle(thisPath)
    data = data.iloc[:, 1:]
    data = data.sort_values(by=['order', 'innerSeq'])
    if 'mrstaat' in data.columns:
        data["mrstaat"] = data.groupby(['order'])['mrstaat'].transform('first')
        data["mrstauc"] = data.groupby(['order'])['mrstauc'].transform('first')
        data["mrsb90"] = data.groupby(['order'])['mrsb90'].transform('first')
        data["mrss90"] = data.groupby(['order'])['mrss90'].transform('first')
        data["aaa"] = data.groupby(['order'])['aaa'].transform('first')
    data["ars"] = data.groupby(['order'])['ars'].transform('first')
    data["sdd"] = data.groupby(['order'])['sdd'].transform('first')
    
    data = data[data['updateType'] == 4]
    data = data[data['secid'].isin(add['secid'].unique())]
    print('1. Loading Data ' + thisDate )
    if data[data['absFilledThisUpdate'] == 0].shape[0] != 0:
        print('There are stocks with zero trade size')
        print(data[data['absFilledThisUpdate'] == 0].groupby(['colo', 'accCode'])['secid'].size())
        data = data[data['absFilledThisUpdate'] != 0]
    if data[data['beta_60'].isnull()].shape[0] != 0:
        print('There are stocks with null beta')
        print(data[data['beta_60'].isnull()])
        data = data[~data['beta_60'].isnull()]
    data['max_trade'] = data.groupby('order')['absOrderSizeCumFilled'].transform('max')
    data['last'] = 0
    data.loc[data['max_trade'] == data['absOrderSizeCumFilled'], 'last'] = 1

    data["buyRet"] = np.where(data["orderDirection"].isin([1, 2]), data["adjMid_F90s"] / data["tradePrice"] - 1, np.nan)
    data["buyRet1"] = np.where(data["orderDirection"].isin([1, 2]), data["adjMid_F300s"] / data["tradePrice"] - 1, np.nan)
    data["sellRet"] = np.where(data["orderDirection"].isin([-1, -2]), data["tradePrice"] / data["adjMid_F90s"] - 1, np.nan)
    data["sellRet1"] = np.where(data["orderDirection"].isin([-1, -2]), data["tradePrice"] / data["adjMid_F300s"] - 1, np.nan)
    data["indexRet"] = data["indexClose_F90s"]/data["indexClose"] - 1
    data["indexRet1"] = data["indexClose_F300s"]/data["indexClose"] - 1
    data["buyAlpha"] = np.where(data["orderDirection"].isin([1, 2]), data["buyRet"] - data["indexRet"] * data["beta_60"], np.nan)
    data["buyAlpha1"] = np.where(data["orderDirection"].isin([1, 2]), data["buyRet1"] - data["indexRet1"] * data["beta_60"], np.nan)
    data["sellAlpha"] = np.where(data["orderDirection"].isin([-1, -2]), data["sellRet"] + data["indexRet"] * data["beta_60"], np.nan)
    data["sellAlpha1"] = np.where(data["orderDirection"].isin([-1, -2]), data["sellRet1"] + data["indexRet1"] * data["beta_60"], np.nan)
    data["buyNum"] = np.where((data["orderDirection"].isin([1, 2])) & (data['last'] == 1), 1, 0)
    data["sellNum"] = np.where((data["orderDirection"].isin([-1, -2])) & (data['last'] == 1), 1, 0)
    data['server_account'] = data['colo'].apply(lambda x: x.replace('_', '')) + '_' + data['accCode'].astype(str)
    df += [data]
df = pd.concat(df, sort=False)

print('2. Orders with nan ars in updateType 0')
if df[df['ars'].isnull()].shape[0] != 0:
    print(df[df['ars'].isnull()])
    df = df[~df['ars'].isnull()]
df.loc[(df['date'] <= 20201013) & (df['ars'].isin([121, 131])), 'mrstauc'] = 1000
df.loc[(df['date'] <= 20201013) & (df['ars'].isin([221, 231])), 'mrstauc'] = 2000
df.loc[(df['date'] <= 20201013) & (df['ars'].isin([321, 331])), 'mrstauc'] = 3000
df.loc[(df['date'] <= 20201013) & (~df['ars'].isin([121, 221, 321, 131, 231, 331])), 'mrstauc'] = 0
df.loc[(df['date'] <= 20201013) & (df['mrstauc'].isin([1000, 2000, 3000])), 'mrstaat'] = 3000
df.loc[(df['date'] <= 20201013) & (df['mrstauc'] == 0), 'mrstaat'] = 1000

print('3. Current strategy status')
if 2 in df['mrstauc'].unique():
    print(df[df['mrstauc'] == 2].groupby('mrstaat')['mrstauc'].unique())
    df.loc[df['mrstauc'] == 2, 'mrstauc'] = 0
print(df.groupby('mrstaat')['mrstauc'].unique())

print('4. Strange strategy')
if df[(df["mrstaat"].isnull()) | (df['mrstaat'] == 0) | (df['mrstaat'] == 2000)].shape[0] != 0:
    print('1) null mrstaat value')
    print(df[(df["mrstaat"].isnull())][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'ars']])
    print(df[(df["mrstaat"].isnull())].groupby(['date', 'colo', 'accCode'])['secid'].size())
    print('2) 0 mrstaat value')
    print(df[(df["mrstaat"] == 0)][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'ars']])
    print('3) 2000 mrstaat value')
    print(df[(df["mrstaat"] == 2000)][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'ars']])
df = df[(~df["mrstaat"].isnull()) & (df['mrstaat'] != 0) & (df['mrstaat'] != 2000)]

if 'mrsb90' in df.columns:
    df['m1'] = df['mrstaat'].apply(lambda x: x - (x // 10000) * 10000)
    df['m2'] = df['mrstauc'].apply(lambda x: x - (x // 10000) * 10000)
    if df[df['mrsb90'] == '-'].shape[0] != 0:
        print('There are ticks with mrsb90 value -')
        print(df[df['mrsb90'] == '-'][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'aaa', 'ars']])
        df = df[df['mrsb90'] != '-']
    if df[df['aaa'] == '-'].shape[0] != 0:
        print('There are ticks with aaa value -')
        print(df[df['aaa'] == '-'][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'aaa', 'ars']])
        df = df[df['aaa'] != '-']   
    df['mrsb90'] = df['mrsb90'].astype(float)
    df['aaa'] = df['aaa'].astype(float)
    df.loc[(df['orderDirection'] >= 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrsb90']) < 1e-12), 'mrstauc'] = \
    df.loc[(df['orderDirection'] >= 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrsb90']) < 1e-12), 'm2']

    df.loc[(df['orderDirection'] >= 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrsb90']) < 1e-12), 'mrstaat'] = \
    df.loc[(df['orderDirection'] >= 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrsb90']) < 1e-12), 'm1']

    df.loc[(df['orderDirection'] < 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrss90']) < 1e-12), 'mrstauc'] = \
    df.loc[(df['orderDirection'] < 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrss90']) < 1e-12), 'm2']

    df.loc[(df['orderDirection'] < 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrss90']) < 1e-12), 'mrstaat'] = \
    df.loc[(df['orderDirection'] < 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrss90']) < 1e-12), 'm1']   

df = pd.merge(df, add[['date', 'secid', 'index_name']], on=['date', 'secid'], how='left')

1. Loading Data 20201201
There are stocks with zero trade size
colo      accCode
zs_88_04  8924       1
Name: secid, dtype: int64
1. Loading Data 20201202
2. Orders with nan ars in updateType 0
3. Current strategy status
mrstaat
1000.0                                                 [0.0]
3000.0                              [1000.0, 2000.0, 3000.0]
11000.0                          [10000.0, 20000.0, 30000.0]
13000.0    [11000.0, 31000.0, 22000.0, 21000.0, 12000.0, ...
Name: mrstauc, dtype: object
4. Strange strategy


C:\Users\win\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [93]:
df3 = df[df['secid'].isin(CSI1000_group1 + CSI1000_group2)]
rdf = df3[df3['accCode'] == 5456]
rdf['order_type'] = np.where(rdf['ars'] == 1, 'take', np.where(
rdf['ars'] == 2, 'improved-NBBO make', 'on-NBBO make'))
rdf['duration'] = np.where(rdf['date'] == 20201201, '20201201-20201201', '20201202-20201202')
rdf['tradeNtl'] = rdf['tradePrice']*rdf['absFilledThisUpdate']
rdf['Side'] = np.where(rdf['orderDirection'] == 1, 'Buy', 'Sell')
rdf['F90s_return'] = np.where(rdf['Side'] == 'Buy', rdf['buyRet'], rdf['sellRet'])
rdf['F300s_return'] = np.where(rdf['Side'] == 'Buy', rdf['buyRet1'], rdf['sellRet1'])

rdf['sumNtl'] = rdf.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['tradeNtl'].transform(sum)
rdf['sum_return'] = rdf['F90s_return'] * rdf['tradeNtl']
rdf['sum_return'] = rdf.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['sum_return'].transform(sum)
rdf['F90s_return'] = rdf['sum_return'] / rdf['sumNtl']
rdf['sum_return1'] = rdf['F300s_return'] * rdf['tradeNtl']
rdf['sum_return1'] = rdf.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['sum_return1'].transform(sum)
rdf['F300s_return'] = rdf['sum_return1'] / rdf['sumNtl']
ff = rdf.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['F90s_return'].first().reset_index()
f = pd.merge(f, ff, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
ff = rdf.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['F300s_return'].first().reset_index()
f = pd.merge(f, ff, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
HTML(f.groupby(['duration', 'Side',  'index_name', 'accCode', 'order_type']).first().to_html())

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [38]:
rdf[(rdf['ars'] == 3) & (rdf['Side'] == 'Sell')]['sellRet'].describe()*10000

count    460000.000000
mean          4.588213
std          24.459948
min         -81.906095
25%          -4.356799
50%           4.453021
75%          15.500324
max          53.444150
Name: sellRet, dtype: float64

In [40]:
rdf[(rdf['ars'] == 3) & (rdf['Side'] == 'Sell')]['tradeNtl'].describe()

count        46.000000
mean      11842.739130
std       18440.466495
min        1589.000000
25%        3296.500000
50%        9697.000000
75%       11387.250000
max      117720.000000
Name: tradeNtl, dtype: float64

In [39]:
rdf[(rdf['ars'] == 2) & (rdf['Side'] == 'Sell')]['sellRet'].describe()*10000

count    200000.000000
mean          3.747541
std          26.769662
min         -57.197988
25%          -1.179508
50%          13.193924
75%          19.746507
max          30.210381
Name: sellRet, dtype: float64

In [50]:
rdf[(rdf['ars'] == 2) & (rdf['Side'] == 'Sell') & (rdf['sellRet'] < -0.0057)]

,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,ars,caamd,cancellationPending,cfe,clock,clockAtArrival,cumSharesBought,cumSharesBuyInserted,cumSharesSellInserted,cumSharesSold,date,finalState,gfe,hee,insertedShortOrder,insertionPending,internalId,inv_L,inv_L0,inv_S,inv_S0,l4tr,locateShares,locateSharesTotal,mfe,mra100,mrb100,mrm,mrm25,mrmum,mrrlma,mrsb300,mrsb90,mrss300,mrss90,mrstaat,mrstauc,mrstaum,mrv,ms,mse,mt,mta,mv,orderDirection,orderId,orderOutstanding,orderPrice,orderSysId,sdd,secid,sequenceNo,session,threadId,totalActions,totalCanceled,tradeId,tradePrice,underlyingIndex,updateType,vai,zipFile,colo,test,order,firstUpdateType,insertNum,innerSeq,firstClock,indexCat,beta_60,mdStartClock,mdClockAtArrival,adjMid_F30s,adjMid_F90s,adjMid_F300s,indexClose,indexClose_F30s,indexClose_F90s,indexClose_F300s,max_trade,last,buyRet,buyRet1,sellRet,sellRet1,indexRet,indexRet1,buyAlpha,buyAlpha1,sellAlpha,sellAlpha1,buyNum,sellNum,server_account,m1,m2,index_name,order_type,duration,tradeNtl,Side,F90s_return,F300s_return,sumNtl,sum_return,sum_return1
79551,0.000987,200.0,200.0,200.0,5456,2.0,1.606787e+15,0.0,786017.0906,1.606787e+15,1.606787e+15,0.0,0.0,800.0,400.0,20201201,1.0,786016.0906,-1.0,0.0,0.0,118.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,14424.0,14389.0,-0.001001,0.000519,1.0,-0.001187,-1.0,-0.002047,-1.0,-0.001506,1000.0,0.0,0.0,59600.0,09:35:52.030187,0.0,-1.0,-1.0,4843356.53,-1.0,868900387.0,0.0,144.28,NaN,93500320.0,2300613.0,11390198.0,0,14023.0,128.0,30.0,1.01e+14,144.28,852,4.0,59000.0,logs_20201201_zs_54_01_day_5456,zs_54_01,53090,53088,0.0,1,654878,1.606787e+15,1000852,1.587835,1.606787e+15,1.606787e+15,144.189167,145.11,143.49,6611.0734,6616.0573,6623.8237,6636.132,200.0,1,NaN,NaN,-0.00572,0.005506,0.001929,0.00379,NaN,NaN,-0.002657,0.011524,0,1,zs5401_5456,1000.0,0.0,CSI1000,improved-NBBO make,1201-1201,28856.0,Sell,0.000207,-0.001991,496358.2,102.544775,-988.411873


In [51]:
rdf[(rdf['ars'] == 3) & (rdf['Side'] == 'Sell') & (rdf['sellRet'] < -0.0081)]

,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,ars,caamd,cancellationPending,cfe,clock,clockAtArrival,cumSharesBought,cumSharesBuyInserted,cumSharesSellInserted,cumSharesSold,date,finalState,gfe,hee,insertedShortOrder,insertionPending,internalId,inv_L,inv_L0,inv_S,inv_S0,l4tr,locateShares,locateSharesTotal,mfe,mra100,mrb100,mrm,mrm25,mrmum,mrrlma,mrsb300,mrsb90,mrss300,mrss90,mrstaat,mrstauc,mrstaum,mrv,ms,mse,mt,mta,mv,orderDirection,orderId,orderOutstanding,orderPrice,orderSysId,sdd,secid,sequenceNo,session,threadId,totalActions,totalCanceled,tradeId,tradePrice,underlyingIndex,updateType,vai,zipFile,colo,test,order,firstUpdateType,insertNum,innerSeq,firstClock,indexCat,beta_60,mdStartClock,mdClockAtArrival,adjMid_F30s,adjMid_F90s,adjMid_F300s,indexClose,indexClose_F30s,indexClose_F90s,indexClose_F300s,max_trade,last,buyRet,buyRet1,sellRet,sellRet1,indexRet,indexRet1,buyAlpha,buyAlpha1,sellAlpha,sellAlpha1,buyNum,sellNum,server_account,m1,m2,index_name,order_type,duration,tradeNtl,Side,F90s_return,F300s_return,sumNtl,sum_return,sum_return1
79424,0.00075,500.0,500.0,500.0,5456,3.0,1.606787e+15,0.0,482722.2118,1.606787e+15,1.606787e+15,1100.0,2200.0,1000.0,500.0,20201201,1.0,482721.2118,-1.0,0.0,0.0,305.0,1100.0,1100.0,0.0,0.0,0.0,0.0,0.0,-1.0,2789.0,2788.0,-0.00037,0.002262,1.0,-0.00179,-1.0,-0.001497,-1.0,-0.000652,1000.0,0.0,0.0,401200.0,09:40:22.242211,0.0,-1.0,-1.0,4997585.49,-1.0,868900975.0,0.0,27.89,NaN,93942440.0,2002918.0,17248127.0,0,14023.0,310.0,78.0,1.02e+14,27.89,852,4.0,400300.0,logs_20201201_zs_54_01_day_5456,zs_54_01,52917,52917,0.0,1,655376,1.606787e+15,1000852,0.75452,1.606787e+15,1.606787e+15,28.035333,28.120323,28.054545,6637.3654,6636.732,6638.0176,6629.7632,500.0,1,NaN,NaN,-0.008191,-0.005865,0.000098,-0.001145,NaN,NaN,-0.008116,-0.006729,0,1,zs5401_5456,1000.0,0.0,CSI1000,on-NBBO make,1201-1201,13945.0,Sell,0.001632,0.000058,544766.0,889.122309,31.585074


In [42]:
rdf[(rdf['ars'] == 2) & (rdf['Side'] == 'Sell')]['tradeNtl'].describe()

count        20.000000
mean      24817.910000
std       30956.752815
min        1621.000000
25%        7224.400000
50%       18681.000000
75%       28466.000000
max      142736.000000
Name: tradeNtl, dtype: float64

In [94]:
for i in ['fill_rate', 'cancel_rate']:
    f[i] = f[i].apply(lambda x: '%.1f%%'%(x*100))
for i in ['orderNtl', 'tradeNtl']:
    f[i] = (f[i]/1000).astype(int)
for i in ['cancel_duration_med', 'cancel_duration_90p']:
    f[i] = (f[i]/1000000).apply(lambda x: '%.0f'%(x))
for i in ['mrm_return', 'F90s_return', 'F300s_return']:
     f[i] = f[i].apply(lambda x: '%.2f'%(x*10000))
f

,duration,Side,order_type,accCode,index_name,orderNtl,tradeNtl,fill_rate,cancel_rate,cancel_duration_med,cancel_duration_90p,mrm_return,F90s_return,F300s_return
0,20201201-20201201,Buy,improved-NBBO make,5456,CSI1000,1673,491,29.3%,74.5%,10,57,8.72,13.87,32.70
1,20201201-20201201,Buy,on-NBBO make,5456,CSI1000,1305,417,32.0%,77.3%,12,51,6.43,16.98,30.40
2,20201201-20201201,Buy,take,5456,CSI1000,710,617,86.8%,8.6%,1,1,0.72,18.36,45.96
3,20201201-20201201,Sell,improved-NBBO make,5456,CSI1000,1477,496,33.6%,67.3%,11,58,16.75,2.07,-19.91
4,20201201-20201201,Sell,on-NBBO make,5456,CSI1000,1563,544,34.8%,64.4%,18,102,17.64,16.32,0.58
5,20201201-20201201,Sell,take,5456,CSI1000,885,830,93.7%,3.8%,1,3,5.92,7.45,-15.81
6,20201202-20201202,Buy,improved-NBBO make,5456,CSI1000,1964,495,25.2%,77.1%,5,31,9.84,0.88,-36.50
7,20201202-20201202,Buy,on-NBBO make,5456,CSI1000,2701,582,21.6%,80.4%,15,52,15.77,9.66,0.81
8,20201202-20201202,Buy,take,5456,CSI1000,582,561,96.5%,4.7%,1,1,2.19,2.87,2.41
9,20201202-20201202,Sell,improved-NBBO make,5456,CSI1000,1939,532,27.5%,74.6%,6,77,17.03,12.62,-6.16


In [95]:
f['stockGroup'] = np.where(f['order_type'].isin(['improved-NBBO make', 'on-NBBO make']), 'groupA', 'groupB')
f = f.rename(columns={'order_type':'orderType', 'fill_rate':'fillRate', 'cancel_rate':'cancelRate', 'mrm_return':'Mta'})
HTML(f.groupby(['duration', 'Side',  'index_name', 'accCode', 'stockGroup', 'orderType'])['orderNtl', 'tradeNtl', 'cancelRate', 'cancel_duration_med', 'cancel_duration_90p', 'fillRate', 'Mta', 'F90s_return', 'F300s_return'].first().to_html())

In [111]:
rdf[(rdf['date'] == 20201202) & (rdf['ars'] == 2) & (rdf['Side'] == 'Buy')].groupby('secid')['F300s_return'].describe() * 10000

,count,mean,std,min,25%,50%,75%,max
secid,,,,,,,,
2002459.0,50000.0,-66.619632,0.000000,-66.619632,-66.619632,-66.619632,-66.619632,-66.619632
2002597.0,20000.0,3.214476,12.955816,-5.946669,-1.366096,3.214476,7.795049,12.375622
2002605.0,10000.0,77.494139,NaN,77.494139,77.494139,77.494139,77.494139,77.494139
2002918.0,50000.0,1.535267,24.872094,-35.093200,-3.427005,-3.427005,24.811773,24.811773
2002959.0,10000.0,-102.416841,NaN,-102.416841,-102.416841,-102.416841,-102.416841,-102.416841
2300073.0,30000.0,-72.495131,0.000000,-72.495131,-72.495131,-72.495131,-72.495131,-72.495131
2300676.0,10000.0,11.764481,NaN,11.764481,11.764481,11.764481,11.764481,11.764481
2300709.0,20000.0,-14.363545,53.293394,-52.047665,-33.205605,-14.363545,4.478516,23.320576
2300788.0,10000.0,28.666193,NaN,28.666193,28.666193,28.666193,28.666193,28.666193


In [118]:
rdf[(rdf['date'] == 20201202) & (rdf['ars'] == 2) & (rdf['Side'] == 'Buy')]['sdd']

332263     93959170.0
332264     93959170.0
332265     93959170.0
332266     93959170.0
332267     93959170.0
332292     95733490.0
332293     95738540.0
332295     93653770.0
332354     93212560.0
332355     93212560.0
332357     93323730.0
332358     93323730.0
332362     94731150.0
332364     93710920.0
332393     94737560.0
332394     94737560.0
332395     94737560.0
332506    102615620.0
332521     93212220.0
332522     94015620.0
332544    100613860.0
Name: sdd, dtype: float64

In [112]:
rdf[(rdf['date'] == 20201202) & (rdf['ars'] == 2) & (rdf['Side'] == 'Buy')].groupby('secid')['tradeNtl'].sum()

secid
2002459.0     82575.0
2002597.0     45025.0
2002605.0     16752.0
2002918.0     55422.0
2002959.0     44052.0
2300073.0    143251.0
2300676.0     78414.0
2300709.0     24924.0
2300788.0      4690.0
Name: tradeNtl, dtype: float64